## Importing required libraries

In [46]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pandas_profiling import ProfileReport
import re 
import warnings
warnings.filterwarnings("ignore")

## Loading and viewing data

In [47]:
imdb_movies = pd.read_csv('Data/movies_metadata.csv')
imdb_movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [48]:
ratings= pd.read_csv('Data/title.ratings.tsv',delimiter="\t")
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1647
1,tt0000002,6.1,198
2,tt0000003,6.5,1348
3,tt0000004,6.2,120
4,tt0000005,6.2,2134


In [49]:
all_movie = imdb_movies.merge(ratings,how='left',left_on='imdb_id',right_on='tconst')
all_movie

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,tconst,averageRating,numVotes
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,tt0114709,8.3,867115.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,tt0113497,7.0,300094.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,tt0113228,6.7,23789.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,tt0114885,5.9,9379.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,tt0113041,6.1,33667.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,tt6209470,6.1,1570.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,tt2028550,7.1,193.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,tt0303758,4.3,858.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,[],Released,NaN,Satan Triumphant,False,0.0,0.0,tt0008536,6.9,147.0


## Explantory Data Analysis

In [50]:
# Extract the columns of interest and replace the NaN with blank string

movies_data = all_movie[['title','genres','overview','production_companies','original_language','status','averageRating','numVotes']].replace(np.nan, '', regex=True)

## Data Cleaning

In [51]:
#Remove movies with less than 5 imdb rating and langauges other than english and hindi

#First replace Empty values "" with 0
movies_data['averageRating']=movies_data['averageRating'].replace("",0)

#movies with rating more than 5 will be included in the list
movie_more5 = movies_data[movies_data['averageRating']>7]

#Only released movies to be included in the list
movie_rel = movie_more5[movie_more5['status']=='Released']

#Convert the title to lowercase
movie_rel['title'] = movie_rel['title'].str.lower()

#Select the major languages
movie_lang = movie_rel[movie_rel['original_language'].isin(['en','ja','it','de','es','ru','ko','hi'])== True]

#Select movies with more than 500 vote counts
movie_vote = movie_lang[movie_lang['numVotes']>10000]

#Reset index after removal of rows
movie_dataset = movie_vote.reset_index().drop(columns='index',axis=1)

In [52]:
movie_dataset

,title,genres,overview,production_companies,original_language,status,averageRating,numVotes
0,toy story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]",en,Released,8.3,867115
1,heat,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","Obsessive master thief, Neil McCauley leads a ...","[{'name': 'Regency Enterprises', 'id': 508}, {...",en,Released,8.2,564175
2,goldeneye,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",James Bond must unmask the mysterious head of ...,"[{'name': 'United Artists', 'id': 60}, {'name'...",en,Released,7.2,235120
3,balto,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",An outcast half-wolf risks his life to prevent...,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",en,Released,7.1,39026
4,nixon,"[{'id': 36, 'name': 'History'}, {'id': 18, 'na...",An all-star cast powers this epic look at Amer...,"[{'name': 'Hollywood Pictures', 'id': 915}, {'...",en,Released,7.1,28373
...,...,...,...,...,...,...,...,...
2806,the keepers,"[{'id': 80, 'name': 'Crime'}, {'id': 99, 'name...",A seven-part docuseries about the unsolved mur...,"[{'name': 'Netflix', 'id': 59811}, {'name': 'F...",en,Released,8.1,17207
2807,what the health,"[{'id': 99, 'name': 'Documentary'}]",What the Health is a ground breaking feature l...,"[{'name': 'Spark Media', 'id': 10081}]",en,Released,7.4,26366
2808,wind river,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",An FBI agent teams with the town's veteran gam...,"[{'name': 'Thunder Road Pictures', 'id': 3528}...",en,Released,7.7,196301
2809,shot caller,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",A newly-released prison gangster is forced by ...,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...",en,Released,7.3,69039


In [53]:
#Define a function to combine all strings

def combine_features(row):
    try:
        return row['title'] +2*(" "+row["genres"])+" "+row["overview"]+" "+row["production_companies"]+3*(" "+row["original_language"])
    except:
        print("Error:",row)

In [54]:
#Combine title, genre, overview and production companies
#Thus a user will be recommended movies that are similar in genre, overiew and production company

movie_dataset["combined_features"] = movie_dataset.apply(combine_features,axis=1)

#Convert the string to lower case
movie_dataset["combined_features"]= movie_dataset["combined_features"].str.lower()

#Lets view one of the combine feature
movie_dataset["combined_features"][0]

"toy story [{'id': 16, 'name': 'animation'}, {'id': 35, 'name': 'comedy'}, {'id': 10751, 'name': 'family'}] [{'id': 16, 'name': 'animation'}, {'id': 35, 'name': 'comedy'}, {'id': 10751, 'name': 'family'}] led by woody, andy's toys live happily in his room until andy's birthday brings buzz lightyear onto the scene. afraid of losing his place in andy's heart, woody plots against buzz. but when circumstances separate buzz and woody from their owner, the duo eventually learns to put aside their differences. [{'name': 'pixar animation studios', 'id': 3}] en en en"

In [55]:
#Lets remove the unwanted parentheses, id, name, punctuation

movie_dataset.astype(str)['combined_features'].map(lambda x:  type(x))

replace_val = ["{'id'","'name':","'},","[","]","'",",","}]","{","}",".",":","'",'[0-9]','id','--']

for i in replace_val:
  movie_dataset["combined_features"] = movie_dataset["combined_features"].str.replace(i, '')

for i in range(len(movie_dataset)):
    movie_dataset["combined_features"][i] = re.sub(' +', ' ', movie_dataset["combined_features"][i]) 

movie_dataset["combined_features"][0]

'toy story animation comedy family animation comedy family led by woody andys toys live happily in his room until andys birthday brings buzz lightyear onto the scene afra of losing his place in andys heart woody plots against buzz but when circumstances separate buzz and woody from their owner the duo eventually learns to put ase their differences pixar animation studios en en en'

In [56]:
#Copy movie dataset to df
df= movie_dataset.copy(deep=True)

#Create a column named index
df['index'] = df.index

## Save the Pre=processed dataset

In [57]:
df

,title,genres,overview,production_companies,original_language,status,averageRating,numVotes,combined_features,index
0,toy story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]",en,Released,8.3,867115,toy story animation comedy family animation co...,0
1,heat,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","Obsessive master thief, Neil McCauley leads a ...","[{'name': 'Regency Enterprises', 'id': 508}, {...",en,Released,8.2,564175,heat action crime drama thriller action crime ...,1
2,goldeneye,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",James Bond must unmask the mysterious head of ...,"[{'name': 'United Artists', 'id': 60}, {'name'...",en,Released,7.2,235120,goldeneye adventure action thriller adventure ...,2
3,balto,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",An outcast half-wolf risks his life to prevent...,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",en,Released,7.1,39026,balto family animation adventure family animat...,3
4,nixon,"[{'id': 36, 'name': 'History'}, {'id': 18, 'na...",An all-star cast powers this epic look at Amer...,"[{'name': 'Hollywood Pictures', 'id': 915}, {'...",en,Released,7.1,28373,nixon history drama history drama an all-star ...,4
...,...,...,...,...,...,...,...,...,...,...
2806,the keepers,"[{'id': 80, 'name': 'Crime'}, {'id': 99, 'name...",A seven-part docuseries about the unsolved mur...,"[{'name': 'Netflix', 'id': 59811}, {'name': 'F...",en,Released,8.1,17207,the keepers crime documentary drama crime docu...,2806
2807,what the health,"[{'id': 99, 'name': 'Documentary'}]",What the Health is a ground breaking feature l...,"[{'name': 'Spark Media', 'id': 10081}]",en,Released,7.4,26366,what the health documentary documentary what t...,2807
2808,wind river,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",An FBI agent teams with the town's veteran gam...,"[{'name': 'Thunder Road Pictures', 'id': 3528}...",en,Released,7.7,196301,wind river action crime mystery thriller actio...,2808
2809,shot caller,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",A newly-released prison gangster is forced by ...,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...",en,Released,7.3,69039,shot caller drama crime thriller drama crime t...,2809


In [58]:
df.to_csv("Appdata/Movie_dataset.csv")

## Create model and predict

In [18]:
#Create count matrix from this new combined column

cv = CountVectorizer()

count_matrix = cv.fit_transform(df["combined_features"])

#Compute the Cosine Similarity based on the count_matrix

cosine_sim = cosine_similarity(count_matrix) 
movie_user_likes = "heat"

In [19]:
#Helper functions to obtain index of movie from its title

def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

In [20]:
#Get index of this movie from its title

movie_index = df[df.title == movie_user_likes]["index"].values[0]

similar_movies =  list(enumerate(cosine_sim[movie_index]))


#Get a list of similar movies in descending order of similarity score

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)

# Print titles of first 5 movies
i=0
for element in sorted_similar_movies:
    if df[df.index == element[0]]["title"].values[0]==movie_user_likes:
        pass
    else:
        print(df[df.index == element[0]]["title"].values[0],element[1])
    i=i+1
    if i>5:
        break

the way of the gun 0.5492301703919119
mystic river 0.5249179879710688
the iceman 0.523966186665859
a most violent year 0.5123183204211359
the falcon and the snowman 0.5085815349770862


In [21]:
sorted_similar_movies

[(4, 0.9999999999999989),
 (1578, 0.5492301703919119),
 (2368, 0.5249179879710688),
 (4532, 0.523966186665859),
 (5005, 0.5123183204211359),
 (1302, 0.5085815349770862),
 (2099, 0.5070068446386043),
 (5191, 0.5037527684340762),
 (3353, 0.5034091834662193),
 (5552, 0.4981063798384225),
 (1537, 0.49651012754757007),
 (3340, 0.49167934822937776),
 (645, 0.49121208482669276),
 (2343, 0.4903743699767512),
 (1731, 0.4893273992174107),
 (2867, 0.48457091858307627),
 (3530, 0.48414918576289884),
 (4530, 0.4802023527384059),
 (5622, 0.479229497931825),
 (5499, 0.47863877241182545),
 (365, 0.4786111046310913),
 (4892, 0.47562067037851796),
 (5264, 0.47537602972796417),
 (1332, 0.47469081480608694),
 (4363, 0.47223967355395985),
 (1128, 0.47222472623567385),
 (5190, 0.47212637681750097),
 (1113, 0.4718902545837686),
 (678, 0.470700597269485),
 (756, 0.470700597269485),
 (1813, 0.4702916479331402),
 (4997, 0.47011185332766847),
 (4440, 0.4697817809641511),
 (2778, 0.4683850630866784),
 (2590, 0.46